# Spelling Correction and Text Segmentation

### Author : Peter Norvig, Afreen Ferdoash

### Prerequisites
 * Python 2.7

### Description
This project aims to perform spelling correction and text segmentation on a user query. By spelling correction, we mean predicting the correct word from an incorrectly typed word. By text segmentation, we mean, the ability to parse an incomprehensible word into meaningful words. 

#### The steps involved in spell-correct are -:
1. Read in a large corpus of words
2. Count the number of times of appearance of each word
3. Generate candidate words from the input word that are -:
    1.  Input word itself
    2. Words that are 1 edit distance (by way of insert, delete, transpose, replace) away
    3. Words that are 2 edit distance (by way of insert, delete, transpose, replace) away
4. Find the candidate word with the maximum probability of occurring in the corpus


#### The steps involved in text segmenation are -:
1. Calculate probability distribution of words in the corpus
2. Calculate probability of occurrance of a sequence of words obtained by a candidate segmentation
3. Return segmentation which maximizes the above probability

In [1]:
import os
os.chdir(r'D:\Desktop\Spell_Correct')

import re
import math
import string
from collections import Counter
from __future__ import division

## Building word list

Text files from the following sources were used to build a list of words and their respective counts of occurance in the corpus.
* Project Gutenberg books
* Wiktionary (most frequent words)
* British National Corpus (most frequent words)
* Banking Regulations Manual (Bank of Phillipines)
* Banking FAQs (SBI)


In [2]:
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open(r'.\all_corpus.txt').read()))

print "Total number of unique words: %d " % len(WORDS)

Total number of unique words: 33695 


## Spelling Correction

The input word is first split into possible pairs of words. A set of candidate words are generated from these pairs by performing deletion, transposition, replacement, insertion at edit distance 1 and edit distance 2. The candidates are then checked for their presence in the corpus and that word is chosen which has the maximum probability of occurrance in the corpus. 
Please note however, that the input word is preferred to candidate words at edit distance 1 which in turn is preferred over candidates at edit distance 2 away.

In [3]:
def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'


def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): return set(w for w in words if w in WORDS)

def correction(word):
    "Find the best spelling correction for word."
    candidates = (known([word]) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=WORDS.get)

### Spelling correction with case and punctuation preservation

The following is just a minor modification to preserve the case and punctuation in a sentence while performing the spelling correction for each word comprising the sentence.

In [4]:
def correct_text(text):
    "Correct all the words within a text, returning the corrected text."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return case_of(word)(correction(word.lower()))

def case_of(text):
    "Return the case-function appropriate for text: upper, lower, title, or just str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [5]:
correct_text('I hav lost my dbit crad....Pleeze hlp me!')

'I had lost my debit card....Please help me!'

## Text Segmentation

We make a segmentation into a first word and remaining characters. If we assume words are independent then we can maximize the probability of the first word adjoined to the best segmentation of the remaining characters.
For this, we first calculate the probability of occurrance of each word in the corpus as the count of its occurance divided by the total number of words in the corpus. For calculating the probability of occurance for a a sequence of words, using the bag of words model assumption, we can compute the product of probability of occurrance of each word in the sequence.

In [6]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(WORDS)

def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())  

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

In [7]:
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [8]:
def segment_text(text):
    return re.sub('[a-zA-Z]+', segment_match, text)

def segment_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return (' '.join(segment(word.lower())))


In [14]:
segment('numbr')

['numb', 'r']

## Performing both Segmentation and Spell-Correction

A Segmentation-first approach worked better than Spell-Correction first approach.Before any text correction, we alter the dictionary of words to retain only the most common one-lettered and two-lettered to avoid 'bad segmentation' (eg. 'numbr' may be segmented to 'numb' + 'r'). 
We first segment each word of the give text and check if all the words in the segmented output belong to the dictionary. If this is the case, we accept the segmentation and move to the next word. If this is not the case we perform spell-correction on the word. 

In [9]:
twowords = [line.rstrip('\n') for line in open(r'.\two-lettered.txt','r+')]
onewords = [line.rstrip('\n') for line in open(r'.\one-lettered.txt','r+')]
allWords = WORDS.keys()
allWords = [word for word in allWords if len(word) > 2]
allWords = allWords + twowords + onewords

In [10]:
def performed_correction(word):
    segmented= segment(word)
    if all(word in allWords for word in segmented ):
        ans = ' '.join(segmented) 
        next
    else:        
        ans = correction(word)
    return ans

In [11]:
def correction_text(text):
    return re.sub('[a-zA-Z]+', correction_match, text)

def correction_match(match):
    word = match.group()
    return performed_correction(word)

In [12]:
correction_text('whatis acvv numbr ? ')

'what is a cvv number ? '